## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [2]:
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [3]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [4]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [5]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [7]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [8]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [9]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')


AIMessage(content='인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고, 이 데이터를 이용하여 일정한 패턴이나 규칙을 학습하는 과정을 말합니다. \n\n일반적으로 인공지능 모델은 입력층, 은닉층, 출력층으로 구성되어 있습니다. 입력층에는 데이터가 들어가고, 은닉층은 데이터를 처리하고, 출력층은 결과를 출력합니다. \n\n인공지능 모델은 데이터와 그에 대한 정답 레이블을 이용하여 학습을 진행합니다. 학습 과정에서 모델은 입력 데이터를 받아들이고, 가중치와 편향을 조정하여 입력 데이터와 정답 레이블 간의 관계를 학습합니다. 이렇게 학습된 모델은 새로운 데이터가 들어왔을 때 정확한 결과를 예측할 수 있습니다.\n\n학습 원리는 주어진 데이터와 정답 레이블을 이용하여 모델을 최적화하는 과정으로, 이를 통해 모델은 데이터에 내재된 패턴이나 규칙을 학습하여 일련의 작업을 수행할 수 있게 됩니다.', response_metadata={'token_usage': {'completion_tokens': 357, 'prompt_tokens': 33, 'total_tokens': 390}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5e1aec2b-842d-47b3-8d85-57efc372c76d-0', usage_metadata={'input_tokens': 33, 'output_tokens': 357, 'total_tokens': 390})

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch 

### 출력파서(Output Parser)


In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [11]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [12]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 데이터를 입력으로 받아서 패턴이나 규칙을 학습하는 과정입니다. 이를 위해서 모델은 가중치(weight)와 편향(bias)이라는 값들을 조정하면서 데이터와 실제 결과의 차이를 최소화하는 방향으로 학습을 진행합니다. \n\n모델은 입력 데이터를 받아들여서 여러 층의 신경망(neural network)을 통해 순전파(feedforward)를 진행하고, 결과를 얻습니다. 이 결과를 실제 결과와 비교하여 오차를 계산하고, 역전파(backpropagation) 알고리즘을 사용하여 이 오차를 최소화하는 방향으로 가중치와 편향을 업데이트 합니다. 이 과정을 여러 번 반복하여 모델이 데이터의 패턴을 학습하게 됩니다.\n\n이렇게 학습된 모델은 이후 새로운 데이터를 입력으로 받았을 때, 학습된 패턴을 기반으로 결과를 예측하거나 분류하는 작업을 수행할 수 있습니다. 이러한 과정을 통해 인공지능 모델은 데이터로부터 유용한 정보를 추출하고 예측하는 능력을 갖추게 됩니다.'

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [13]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4-turbo")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [14]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [15]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

영어 회화:
- Hello, I'd like to have a table for one, please.
- Could I see the menu, please?
- I'm ready to order. I’ll have the grilled salmon with a side of vegetables.
- Could I get a glass of water as well?
- Thank you!

한글 해석:
- 안녕하세요, 혼자 식사할 테이블을 부탁합니다.
- 메뉴판 좀 볼 수 있을까요?
- 주문할게요. 그릴 연어에 야채 사이드를 주세요.
- 물 한 잔도 주실 수 있나요?
- 감사합니다!


In [16]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
print(chain.invoke({"question": "미국에서 피자 주문"}))

영어 회화:
- Hello, I’d like to order a large pepperoni pizza for delivery, please.
- Sure, anything else to add with that?
- Yes, can I also get an order of garlic bread and a 2-liter bottle of Coke?
- Absolutely, can I have your address and phone number, please?
- It's 123 Maple Street, and my phone number is 555-1234.
- Thank you! Your total comes to $28.50. We should have your order delivered in about 30-40 minutes.
- Great, thank you!

한글 해석:
- 안녕하세요, 배달로 대형 페퍼로니 피자 하나 주문하고 싶습니다.
- 네, 그 외에 더 추가하실 것이 있으신가요?
- 네, 마늘 빵 하나와 콜라 2리터도 주문할게요.
- 알겠습니다, 주소와 전화번호를 알려주시겠어요?
- 주소는 메이플 스트리트 123번지고, 전화번호는 555-1234입니다.
- 감사합니다! 총 금액은 $28.50입니다. 주문하신 음식은 대략 30-40분 내에 배달될 예정입니다.
- 좋습니다, 감사합니다!
